# Definitions

In [ ]:
# change working directory to the project root
import os
os.chdir('../../')

import sys
sys.path.append('models/utils')
sys.path.append('models/brian2')
sys.path.append('models/aln')

In [ ]:
# import python packages
from __future__ import print_function
import os
import datetime
import tqdm
import matplotlib.pyplot as plt
% matplotlib inline
import numpy as np
import scipy 
import pandas as pd
import pypet as pp

# import utils libs
import pypet_parameters as pe
import fitparams as fp
import functions as func
import runModels as rm
import paths

In [ ]:
# sett dpi for notebooks on server
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['image.cmap'] = 'plasma'

In [ ]:
# load default parameters
params = []
params = fp.loadpoint(params, "A1")
params['sigma_ou'] = 0.0 # no mean current fluctuations
params['dt'] = 0.1 # timestep for simulations
params['duration'] =  6000 # ms

# Number of neurons for AdEx network simulation
N_neurons = 10000
params['model'] = 'brian'
params['N'] = N_neurons

# Parameter exploration

In [ ]:
parametrization_noadapt_aln = pp.cartesian_product({
    'mue_ext_mean':[ round(elem, 3) for elem in np.linspace(0.0, 3.0, 251)],
    'mui_ext_mean':[ round(elem, 3) for elem in np.linspace(0.0, 3.0, 251)],
    'model':[ 'aln'],
    'load_point':[ 'A2' ],
    'duration': [ 6000 ]
})

parametrization_noadapt_brian = pp.cartesian_product({
    'mue_ext_mean':[ round(elem, 3) for elem in np.linspace(0.0, 3.0, 91)],
    'mui_ext_mean':[ round(elem, 3) for elem in np.linspace(0.0, 3.0, 91)],
    'model':['brian'],
    'load_point':[ 'A2' ],
    'duration': [ 6000 ]
})

parametrization_adapt_aln = pp.cartesian_product({
     'mue_ext_mean':[ round(elem, 3) for elem in np.linspace(1.0, 5.0, 251)],
     'mui_ext_mean':[ round(elem, 3) for elem in np.linspace(0.0, 4.0, 251)],
    'model':[ 'aln' ],
    'load_point':[ 'B2' ],
    'duration' : [ 6000 ]
})

parametrization_adapt_brian = pp.cartesian_product({
     'mue_ext_mean':[ round(elem, 3) for elem in np.linspace(1.0, 5.0, 91)],
     'mui_ext_mean':[ round(elem, 3) for elem in np.linspace(0.0, 4.0, 91)],
    'model':[ 'brian' ],
    'load_point':[ 'B2' ],
    'duration' : [ 6000 ]
})

parametrization = parametrization_noadapt_aln

for k in parametrization:
    parametrization[k].extend(parametrization_noadapt_brian[k])
    parametrization[k].extend(parametrization_adapt_aln[k])
    parametrization[k].extend(parametrization_adapt_brian[k])

In [ ]:
print("Number of parameter configurations: {}".format(len(parametrization[parametrization.keys()[0]])))

## Some tests

In [ ]:
# load test parameters
npar = 81

for k in parametrization:
    print(k, parametrization[k][npar])
    params[k] = parametrization[k][npar]

In [ ]:
# test run
t_test, re_test, ri_test, stimulus_test = rm.runModels_stimulus(manual_params=params)

In [ ]:
# plot test run
plt.plot(t_test, re_test)
plt.plot(t_test[-len(stimulus_test):], stimulus_test/np.max(stimulus_test)*np.max(re_test)/4, alpha=0.85, c='tomato')
plt.show()

## Run simulations

In [ ]:
# ---- initialize pypet environment ----
trajectoryName = 'results' + datetime.datetime.now().strftime("-%Y-%m-%d-%HH-%MM-%SS")
HDF_FILE = os.path.join(paths.HDF_DIR, 'bifurcation-diagrams-brian-aln.hdf')
trajectoryFileName = HDF_FILE

import multiprocessing
ncores = multiprocessing.cpu_count()
print("Number of cores: {}".format(ncores))

env = pp.Environment(trajectory=trajectoryName,filename=trajectoryFileName,
                    file_title='E-I model exploration for bifurcation diagrams',
                    large_overview_tables=True,
                    multiproc=True,           
                    ncores=ncores,
                    wrap_mode='QUEUE',
                    log_stdout = False
                    )

# Get the trajectory from the environment
traj = env.v_trajectory
trajectoryName = traj.v_name

pe.add_parameters(traj, params)

In [ ]:
traj.f_explore(parametrization)
env.f_run(rm.runModels_stimulus)
env.f_disable_logging()
print("Done.")

# Data analysis

## Load results from disk

In [ ]:
HDF_FILE = os.path.join(paths.HDF_DIR, 'bifurcation-diagrams-brian-aln.hdf')
trajectoryFileName = HDF_FILE

In [ ]:
# ---- load pypet trajectory "trajectoryFileName" ----
print("Analyzing File \"{}\"".format(trajectoryFileName))
print("All Trajectories:")
print(pe.getTrajectoryNameInsideFile(trajectoryFileName))
trajectoryName = pe.getTrajectoryNameInsideFile(trajectoryFileName)[-1]

print("Analyzing trajectory \"{}\".".format(trajectoryName))
trajLoaded = pp.Trajectory(trajectoryName,add_time=False)
trajLoaded.f_load(trajectoryName,filename=trajectoryFileName, force=True)
trajLoaded.v_auto_load = True
print("{} results found".format(len(trajLoaded.f_get_results())))

## Process data

In [ ]:
nResults = len(trajLoaded.f_get_run_names()) # number of results in trajectory 
dt = trajLoaded.f_get_parameters()['parameters.simulation.dt'].f_get() # dt of simulation

# ---- explored parameter list ----
exploredParameters = trajLoaded.f_get_explored_parameters()
niceParKeys = [p.split('.')[-1] for p in exploredParameters.keys()]

# ---- lcreate pandas df with results as keys ----
dfResults = pd.DataFrame(columns=niceParKeys,dtype=object)

# range of parameters
for nicep, p in zip(niceParKeys,exploredParameters.keys()):
    dfResults[nicep] = exploredParameters[p].f_get_range()

### Serial Processing

In [ ]:
# ---- make a dictionary with results ----
resultDicts = []
makeDict = True

#measures = ['domfr_exc', 'domfr_power_exc', 'max_exc', 'min_exc', 'updowndiff_exc', 'domfr_inh','domfr_power_inh', 'max_inh', 'min_inh', 'updowndiff_inh'] 
measures = ['domfr_exc', 'max_exc', 'updowndiff_exc', 'domfr_power_exc', 'max_inh']

for rInd in tqdm.tqdm(range(nResults), total=nResults):
    
    for measure in measures:
        dfResults.loc[rInd, measure] = func.analyse_run(measure, trajLoaded.results[rInd].f_to_dict(), dt)    
    if makeDict:
            resultDicts.append(trajLoaded.results[rInd].f_to_dict())
            
print("done.")

### Parallel processing

In [ ]:
#print(trajLoaded.f_get('parameters.globalNetwork.N').f_get())
#print(np.unique(trajLoaded.f_get('parameters.simulation.duration').f_get_range()))
#mue_range = trajLoaded.f_get('parameters.localNetwork.mue_ext_mean').f_get_range()
#mui_range = trajLoaded.f_get('parameters.localNetwork.mui_ext_mean').f_get_range()

#print("Range: mue {} to {} ({} steps), mui {} to {} ({} steps)".format(mue_range[0], mue_range[-1], len(np.unique(mue_range)),
#                                                                       mui_range[0], mui_range[-1], len(np.unique(mui_range))))

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()   

# ---- make a dictionary with results ----
resultDicts = []
makeDict = False
print("Creating results dictionary ...")
runResults = []
for rInd in range(len(dfResults)):
    result = trajLoaded.results[rInd].f_to_dict()
    runResults.append(result)
    if makeDict:
        resultDicts.append(result)
print("done.")

for measure in ['domfr_exc', 'max_exc', 'updowndiff_exc', 'domfr_power_exc', 'max_inh']:
    print("Analysing measure: {}".format(measure))
    parallelResults = Parallel(n_jobs=num_cores)(delayed(func.analyse_run)(measure = measure, result = i, dt = dt) for i in runResults)
    dfResults[measure] = parallelResults
print("done.")

## Save analyzed results

In [ ]:
import pickle
file_path_p = os.path.join(paths.PICKLE_DIR, 'pypet-bifurcation-diagram-large-longer-dfResults.p')
if not os.path.isdir(paths.PICKLE_DIR): os.makedirs(paths.PICKLE_DIR)
# uncomment to save results
#pickle.dump(dfResults, file(file_path_p, "w"))

# load precomputed results
#dfResults = pickle.load(file(file_path_p, "r"))

# Plot data

In [ ]:
def annotate_plot(point, model, ax, annotate_points = True):
    C = 1 # Capacitance, convert mV/ms to pA
    if point[0] == 'A':
        if model == 'brian':
            # attractor labels -------------
            ax.text(0.8,0.25, 'up', fontweight='bold', transform=ax.transAxes, #style='italic',
                    bbox={'facecolor':'white', 'alpha':0.85, 'pad':8}, fontsize=25)
            ax.text(0.35,0.06, 'LC$_{EI}$', fontweight='bold', transform=ax.transAxes, #style='italic',
                    bbox={'facecolor':'white', 'alpha':0.85, 'pad':8}, fontsize=25)
            ax.text(0.25,0.65, 'down', fontweight='bold', transform=ax.transAxes, #style='italic',
                    bbox={'facecolor':'white', 'alpha':0.85, 'pad':8}, fontsize=25)
            ax.text(0.8,0.65, 'bi', fontweight='bold', transform=ax.transAxes, #style='italic',
                    bbox={'facecolor':'white', 'alpha':0.85, 'pad':8}, fontsize=25)

        else:
            # attractor labels -------------
            ax.text(2.3,0.75, 'up', fontweight='bold',#style='italic',
                    bbox={'facecolor':'white', 'alpha':0.9, 'pad':8}, fontsize=25)
            ax.text(0.7,0.3, 'LC$_{EI}$', fontweight='bold',#style='italic',
                    bbox={'facecolor':'white', 'alpha':0.9, 'pad':8}, fontsize=25)
            ax.text(0.6,1.75, 'down', fontweight='bold',#style='italic',
                    bbox={'facecolor':'white', 'alpha':0.9, 'pad':8}, fontsize=25)
            ax.text(2.2,2.5, 'bi', fontweight='bold',#style='italic',
                    bbox={'facecolor':'white', 'alpha':0.9, 'pad':8}, fontsize=25)

        if annotate_points:
            for p in ['A1', 'A2', 'A3']:
                if model == 'brian':
                    params_thispoint = fp.loadpoint_network(params, p)
                else: 
                    params_thispoint = fp.loadpoint(params, p)
                plotpoint = (params_thispoint['mue_ext_mean'], params_thispoint['mui_ext_mean'])
                bbox_props = dict(boxstyle="circle", fc="w", ec="0.5", pad=0.2, alpha=0.9)
                ax.text(plotpoint[0], plotpoint[1], p, ha="center", va="center", size=20, bbox=bbox_props, transform=ax.transData)

    elif point[0] == 'B':
        if model == 'brian':
            # attractor labels -------------
            ax.text(0.88,0.25, 'up', fontweight='bold', transform=ax.transAxes, #style='italic',
                    bbox={'facecolor':'white', 'alpha':0.85, 'pad':8}, fontsize=25)
            ax.text(0.5,0.06, 'LC$_{EI}$', fontweight='bold',transform=ax.transAxes, #style='italic',
                    bbox={'facecolor':'white', 'alpha':0.85, 'pad':8}, fontsize=25)
            ax.text(0.3,0.7, 'down', fontweight='bold',transform=ax.transAxes, #style='italic',
                    bbox={'facecolor':'white', 'alpha':0.85, 'pad':8}, fontsize=25)
            ax.text(0.68,0.39, 'LC$_{aE}$', fontweight='bold',transform=ax.transAxes, #style='italic',
                    bbox={'facecolor':'white', 'alpha':0.85, 'pad':8}, fontsize=25)
        else:
            # attractor labels -------------
            ax.text(2.5, 0.3, u'LC$_{EI}$', fontweight='bold',#style='italic',
                    bbox={'facecolor':'white', 'alpha':0.9, 'pad':8}, fontsize=25)
            ax.text(4.0, 2.8, u'LC$_{aE}$', fontweight='bold',#style='italic',
                    bbox={'facecolor':'white', 'alpha':0.9, 'pad':8}, fontsize=25)
            ax.text(1.6, 2.75, 'down', fontweight='bold',#style='italic',
                    bbox={'facecolor':'white', 'alpha':0.9, 'pad':8}, fontsize=25)
            ax.text(4.3, 0.7, 'up', fontweight='bold',#style='italic',
                    bbox={'facecolor':'white', 'alpha':0.9, 'pad':8}, fontsize=25)
            
        if annotate_points:
            for p in ['B3', 'B4']:
                if model == 'brian':
                    params_thispoint = fp.loadpoint_network(params, p)
                else: 
                    params_thispoint = fp.loadpoint(params, p)
                plotpoint = (params_thispoint['mue_ext_mean'], params_thispoint['mui_ext_mean'])
                bbox_props = dict(boxstyle="circle", fc="w", ec="0.5", pad=0.2, alpha=0.9)
                ax.text(plotpoint[0], plotpoint[1], p, ha="center", va="center", size=20, bbox=bbox_props, transform=ax.transData)
            

In [ ]:
up_down_threshold_hz = 10 # minimum difrernce of population rate between up and down state for bistable classification
domfr_threshold = 0.5 # minimum dominant frequency of an oscillation for classification

def plotResults(selectedResults, plot='max_exc', colorbarlabel = 'Maximum rate of E [Hz]', clim_max = 80, annotate = True, annotate_points = True, savefigname = None):
    point = selectedResults.load_point.iloc[0]
    model = selectedResults.model.iloc[0]
    
    plot_results = []
    amplitudes = []
    frequencies = []
    oscillation_powers = []
    max_inh_rates = []

    up_down_differences = []
    
    for i in range(len(selectedResults)):
        plot_results.append(selectedResults.iloc[i][plot])
        #max_inh_rates.append(r['net']['max_inh'])
        #amplitudes.append(selectedResults.iloc[i]['max_exc']-selectedResults.iloc[i]['min_exc'])
        frequencies.append(selectedResults.iloc[i]['domfr_exc'])
        up_down_differences.append(selectedResults.iloc[i]['updowndiff_exc'])
        #oscillation_powers.append(selectedResults.iloc[i]['domfr_power_exc'])
    
    
        
    mues = np.unique(selectedResults['mue_ext_mean'])  
    muis = np.unique(selectedResults['mui_ext_mean'])    
    
    # prepare matrixes for plotting
    plot_results_matrix = np.reshape(plot_results, (len(mues), len(muis))).T
    frequencies_matrix = np.reshape(frequencies, (len(mues), len(muis))).T
    up_down_differences_matrix = np.reshape(up_down_differences, (len(mues), len(muis))).T
    
    # Fill in NaN's...
    mask = np.isnan(plot_results_matrix)
    plot_results_matrix[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), plot_results_matrix[~mask])
    mask = np.isnan(frequencies_matrix)
    frequencies_matrix[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), frequencies_matrix[~mask])
    mask = np.isnan(up_down_differences_matrix)
    up_down_differences_matrix[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), up_down_differences_matrix[~mask])  
    
    # if domfr, then zeros should be nan
    if plot.startswith("domfr"):
        plot_results_matrix = np.ma.masked_where(plot_results_matrix == 0, plot_results_matrix)        
    
    # --------------------------
    # -------  PLOTTING  -------
    # --------------------------
        
    plt.figure(figsize=(10,8), dpi=300)
    ax = plt.gca()
    plt.imshow(plot_results_matrix, origin='lower', aspect='auto', extent = [mues[0], mues[-1], muis[0], muis[-1]])
    
    plt.locator_params(nbins=4 if point[0] == 'A' else 5) # reduce number of ticks
    ax.tick_params(labelsize=20) 
    # multiply axis ticks with capacitance C to get nA 
    ax.set_xticklabels(np.round(np.multiply(ax.get_xticks(),0.2),2)) 
    ax.set_yticklabels(np.round(np.multiply(ax.get_yticks(),0.2),2)) 
    
    plt.xlabel("Input to E [nA]", fontsize=30)
    plt.ylabel("Input to I [nA]", fontsize=30)

    plt.clim(0, clim_max)
    cbar  = plt.colorbar(label='Maximum rate of E [Hz]', extend='max')
    cbar.ax.tick_params(labelsize=15) 
    
    cbar.set_label(colorbarlabel, fontsize = 22)
    
    if annotate:
        # ----- LC contour -----
        Z=frequencies_matrix.copy()
        blurredZ =scipy.ndimage.filters.gaussian_filter(frequencies_matrix,0.0) # blurred
        Z = blurredZ.copy()
        frequency_threshold = domfr_threshold
        Z[Z<frequency_threshold] = 0
        Z[Z>=frequency_threshold] = 1

        Xi,Yi = np.meshgrid(mues, muis)
        cset2 = plt.contour(Xi, Yi, Z, colors='white', linestyles='solid', levels=[0, 1.0001], linewidths=(7,),zorder=1)

        # ----- bistability contour -----
        Z=up_down_differences_matrix.copy()
        blurredZ =scipy.ndimage.filters.gaussian_filter(up_down_differences_matrix,0.0) # blurred
        Z = blurredZ.copy()
        up_down_threshold = up_down_threshold_hz
        Z[Z<up_down_threshold] = 0
        Z[Z>=up_down_threshold] = 1

        Xi,Yi = np.meshgrid(mues, muis)
        levels = np.arange(0,1,1)
        cset2 = plt.contour(Xi, Yi, Z, colors='springgreen', linestyles='dashed', levels=[0, 1.0001], linewidths=(7,),zorder=2)

        # annotate the plot with labels
        annotate_plot(point, model, ax, annotate_points)
    if savefigname:
        fname = savefigname#, colorbarlabel[:-5].replace(" ", "_")
        print("Saving {}".format(fname))
        for extension in ['png', 'svg', 'jpg']:
            plt.savefig(os.path.join(paths.FIGURES_DIR, "bifurcation-plot-{}.{}".format(fname, extension)))
    plt.show()

In [ ]:
load_point_range = np.unique(dfResults.load_point)
model_range = np.unique(dfResults.model)

for point in load_point_range:
    for model in model_range:
        print(point, model)
        selector = (dfResults.model==model) & (dfResults.load_point==point)
        selectedResults = dfResults[selector]
        selectIndices = selectedResults.index
        plotResults(selectedResults, 'max_exc', clim_max=80, colorbarlabel='Maximum rate of E [Hz]', savefigname='max_r_I_{}_{}'.format(model, point[0]))
        plotResults(selectedResults, 'max_inh', clim_max=80, colorbarlabel='Maximum rate of I [Hz]', savefigname='max_r_I_{}_{}'.format(model, point[0]), annotate_points = False)
        plotResults(selectedResults, 'domfr_exc', clim_max=35, colorbarlabel='Dominant frequency of E [Hz]', annotate=False, savefigname='domfr_E_{}_{}'.format(model, point[0]))
        #plotResults(selectedResults, 'domfr_power_exc', clim_max=1000, colorbarlabel='Power of dominant frequency [Hz]', annotate=False)

## Special definitions for plottings difference max(E) - max(I)

In [ ]:
up_down_threshold_hz = 10
domfr_threshold = 0.1

def plotResults_exc_inh_diff(selectedResults, plot='max_exc', colorbarlabel = 'Maximum rate of E [Hz]', clim_max = 80, annotate = True, savefigname = None):
    point = selectedResults.load_point.iloc[0]
    model = selectedResults.model.iloc[0]
    
    plot_results = []
    plot_results_inh = []
    amplitudes = []
    frequencies = []
    oscillation_powers = []
    max_inh_rates = []

    up_down_differences = []
    
    for i in range(len(selectedResults)):
        plot_results.append(selectedResults.iloc[i]['max_exc'])
        plot_results_inh.append(selectedResults.iloc[i]['max_inh'])
        #max_inh_rates.append(r['net']['max_inh'])
        #amplitudes.append(selectedResults.iloc[i]['max_exc']-selectedResults.iloc[i]['min_exc'])
        frequencies.append(selectedResults.iloc[i]['domfr_exc'])
        up_down_differences.append(selectedResults.iloc[i]['updowndiff_exc'])
        #oscillation_powers.append(selectedResults.iloc[i]['domfr_power_exc'])
    
    mues = np.unique(selectedResults['mue_ext_mean'])  
    muis = np.unique(selectedResults['mui_ext_mean'])    
    
    # prepare matrixes for plotting
    plot_results_matrix = np.reshape(plot_results, (len(mues), len(muis))).T
    plot_results_inh_matrix = np.reshape(plot_results_inh, (len(mues), len(muis))).T
    
    frequencies_matrix = np.reshape(frequencies, (len(mues), len(muis))).T
    up_down_differences_matrix = np.reshape(up_down_differences, (len(mues), len(muis))).T
    
    # Fill in NaN's...
    mask = np.isnan(plot_results_matrix)
    plot_results_matrix[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), plot_results_matrix[~mask])
    mask = np.isnan(plot_results_inh_matrix)
    plot_results_inh_matrix[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), plot_results_inh_matrix[~mask])
    #mask = np.isnan(amplitudes_matrix)
    #amplitudes_matrix[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), amplitudes_matrix[~mask])
    mask = np.isnan(frequencies_matrix)
    frequencies_matrix[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), frequencies_matrix[~mask])
    mask = np.isnan(up_down_differences_matrix)
    up_down_differences_matrix[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), up_down_differences_matrix[~mask])  
    
    
    # subtract inhibitory rates from excitatory rates
    plot_results_matrix -= plot_results_inh_matrix
    
    # --------------------------
    # -------  PLOTTING  -------
    # --------------------------
    
    C = 1 # Capacitance, convert mV/ms to pA
    
    plt.figure(figsize=(10,8), dpi=300)
    ax = plt.gca()
    plt.imshow(plot_results_matrix, origin='lower', aspect='auto', extent = [mues[0]*C, mues[-1]*C, muis[0]*C, muis[-1]*C], cmap='seismic')
    
    plt.locator_params(nbins=4 if point[0] == 'A' else 5) # reduce number of ticks
    ax.tick_params(labelsize=20) 
    # multiply axis ticks with capacitance C to get nA 
    ax.set_xticklabels(np.multiply(ax.get_xticks(),0.2)) 
    ax.set_yticklabels(np.multiply(ax.get_yticks(),0.2)) 
        
    plt.xlabel("Input to E [nA]", fontsize=30)
    plt.ylabel("Input to I [nA]", fontsize=30)

    plt.clim(-100, 100)
    cbar  = plt.colorbar(label='Maximum rate of E [Hz]', extend='both')
    cbar.ax.tick_params(labelsize=15) 
    
    cbar.set_label(colorbarlabel, fontsize = 22)
    
    if annotate:
        # ----- LC contour -----
        Z=frequencies_matrix.copy()
        blurredZ =scipy.ndimage.filters.gaussian_filter(frequencies_matrix,0.0) # blurred
        Z = blurredZ.copy()
        frequency_threshold = domfr_threshold
        Z[Z<frequency_threshold] = 0
        Z[Z>=frequency_threshold] = 1

        Xi,Yi = np.meshgrid(mues*C, muis*C)
        cset2 = plt.contour(Xi, Yi, Z, colors='white', linestyles='solid', levels=[0, 1.0001], linewidths=(7,),zorder=1)

        # ----- bistability contour -----
        Z=up_down_differences_matrix.copy()
        blurredZ =scipy.ndimage.filters.gaussian_filter(up_down_differences_matrix,0.0) # blurred
        Z = blurredZ.copy()
        up_down_threshold = up_down_threshold_hz
        Z[Z<up_down_threshold] = 0
        Z[Z>=up_down_threshold] = 1

        Xi,Yi = np.meshgrid(mues*C, muis*C)
        levels = np.arange(0,1,1)
        cset2 = plt.contour(Xi, Yi, Z, colors='springgreen', linestyles='dashed', levels=[0, 1.0001], linewidths=(7,),zorder=2)

        # annotate the plot with labels
        annotate_plot(point, model, ax, annotate_points = False)
    plt.show()

In [ ]:
load_point_range = np.unique(dfResults.load_point)
model_range = np.unique(dfResults.model)

for point in load_point_range:
    for model in model_range:
        print(point, model)
        selector = (dfResults.model==model) & (dfResults.load_point==point)
        selectedResults = dfResults[selector]
        selectIndices = selectedResults.index
        plotResults_exc_inh_diff(selectedResults, 'max_exc', clim_max=80, colorbarlabel='max(E) - max(I) [Hz]')

## Plot individual trace from results

In [ ]:
# load result from hdf trajectory
run_id = 11
res = trajLoaded.results[run_id].f_to_dict() 
t_net, re_net, ri_net, stimulus_net = res['t'], res['rate_exc'], res['rate_inh'], res['stimulus']

# plot
plt.plot(t_net, re_net)
plt.plot(t_net[-len(stimulus_net):], stimulus_net/np.max(stimulus_net)*np.max(re_net)/4, alpha=0.85, c='tomato')
plt.show()